In [1]:
"""
another way to initialize C:
n = len(A)
C = [[0] * n for _ in range(n)]
"""
def matrix_multiply(A, B, C, n):
    for i in range(n):
        for j in range(n):
            C[i][j] = 0
            for k in range(n):
                C[i][j] += C[i][j] + A[i][k] * B[k][j]

In [2]:
def add(X, Y):
    n = len(X)
    return [[X[i][j] + Y[i][j] for j in range(n)] for i in range(n)]

def matrix_multiply_recursive(A, B):
    n = len(A)
    
    if n == 1:
        return [[A[0][0] * B[0][0]]]
        
    # Partition A, B, C into quadrants and recurse
    mid = n // 2

    A11 = [row[:mid] for row in A[:mid]]
    A12 = [row[mid:] for row in A[:mid]]
    A21 = [row[:mid] for row in A[mid:]]
    A22 = [row[mid:] for row in A[mid:]]

    B11 = [row[:mid] for row in B[:mid]]
    B12 = [row[mid:] for row in B[:mid]]
    B21 = [row[:mid] for row in B[mid:]]
    B22 = [row[mid:] for row in B[mid:]]

    C11 = add(
        matrix_multiply_recursive(A11, B11),
        matrix_multiply_recursive(A12, B21),
    )
    
    C12 = add(
        matrix_multiply_recursive(A11, B12),
        matrix_multiply_recursive(A12, B22),
    )

    C21 = add(
        matrix_multiply_recursive(A21, B11),
        matrix_multiply_recursive(A22, B21),
    )

    C22 = add(
        matrix_multiply_recursive(A21, B12),
        matrix_multiply_recursive(A22, B22),
    )

    C = []
    for i in range(mid):
        C.append(C11[i] + C12[i])
    for i in range(mid):
        C.append(C21[i] + C22[i])

    return C

In [3]:
def multiply_cell(A, B, i, j, k_start, k_end):
    if k_start == k_end:
        return A[i][k_start] * B[k_start][j]

    mid = (k_start + k_end) // 2
    return (
        multiply_cell(A, B, i, j, k_start, mid) +
        multiply_cell(A, B, i, j, mid + 1, k_end)
    )

def matrix_mult_recursive(A, B):
    n = len(A)
    C = [[0] * n for _ in range(n)]

    for i in range(n):
        for j in range(n):
            C[i][j] = multiply_cell(A, B, i, j, 0, n-1)

In [4]:
"""
Strassen matrix

This shows us how to get a solution to a matix dot product
faster than O(n^3) -> O(n^lg7) ~ O(n^2.81)
"""

A = [1,3,
     7,5]
B = [6,8,
     4,2]

"""
A11 = 1, A12 = 3, A21 = 7, A22 = 5
B11 = 6, B12 = 8, B21 = 4, B22 = 2
"""

"""
STEP 1:
S1 = B21 - B22 = 8 - 2 = 6
S2 = A11 + A12 = 1 + 3 = 4
S3 = A21 + A22 = 7 + 5 = 12
S4 = B21 - B11 = 4 - 6 = -2
S5 = A11 + A22 = 1 + 5 = 6
S6 = B11 + B22 = 6 + 2 = 8
S7 = A12 - A22 = 3 - 5 = -2
S8 = B21 + B22 = 4 + 2 = 6
S9 = A11 - A21 = 1 - 7 = -6
S10 = B11 + B12 = 6 + 8 = 14
"""

"""
STEP 2:
P1 = A11 * S1 = 1 * 6 = 6
P2 = S2 * B22 = 4 * 2 = 8
P3 = S3 * B11 = 12 * 6 = 72
P4 = A22 * S4 = 5 * (-2) = -10
P5 = S5 * S6 = 6 * 8 = 48
P6 = S7 * S8 = (-2) * 6 = -12
P7 = S9 * S10 = (-6) * 14 = -84
"""

"""
STEP 3:
C11 = P5 + P4 - P2 + P6
    = 48 - 10 - 8 - 12 = 18
C12 = P1 + P2 = 6 + 8 = 14
C21 = P3 + P4 = 72 - 10 = 62
C22 = P5 + P1 - P3 - P7 = 48 + 6 - 72 - (-84) = 66
"""

AB = [18, 14,
      62, 66]

"""
alt Strassen matrix
"""

A = [1,3,
     7,5]
B = [6,8,
     4,2]

"""
STEP 1:
M1 = (a + d)(e + h)
M2 = (c + d)e
M3 = a(f - h)
M4 = d(g - e)
M5 = (a + b)h
M6 = (c - a)(e + f)
M7 = (b - d)(g + h)

A = [a, b,
     c, d]

B = [e, f,
     g, h]

a = 1, b = 3, c = 7, d = 5
e = 6, f = 8, g = 4, h = 2
"""

"""
STEP 2:
M1 = (1 + 5)(6 + 2) = 6 * 8 = 48
M2 = ( 7 + 5) * 6 = 12 * 6 = 72
M3 = 1 * (8 - 2) = 6
M4 = 5 * (4 - 6) = 5 * (-2) = -10
M5 = (1 + 3) * 2 = 4 * 2 = 8
M6 = (7 - 1)(6 + 8) = 6 * 14 = 84
M7 = (3 - 5)(4 + 2) = (-2) * 6 = -12
"""

"""
STEP 3:
C11 = M1 + M4 - M5 + M7 = 48 - 10 - 8 - 12 = 18
C12 = M3 + M5 = 6 + 8 = 14
C21 = M2 + M4 = 72 - 10 = 62
C22 = M1 - M2 + M3 + M6 = 48 - 72 + 6 + 84 = 66
"""

AB = [18, 14,
      62, 66]

In [21]:
"""
Good/Bad Chips
Pair chips arbitrarily: floor(n/2) tests.

Apply elimination rules:

Both say good → keep 1

At least one says bad → discard both

If odd, carry over the extra chip.

Resulting set has:

≤ ceil(n/2) chips

Still a strict majority of good chips

This gives the first step in the recursive reduction used in the “find one good chip” algorithm.
"""
class Chip:
    def __init__(self, chip_id, kind):
        self.id = chip_id
        self.kind = kind # good/bad

    def report_on(self, other):
        return (other.kind == "good") if self.kind == "good" else (other.kind != "good")

def test(chipA, chipB):
    return chipA.report_on(chipB), chipB.report_on(chipA)

chips = [
    Chip(1, "good"),
    Chip(2, "bad"),
    Chip(3, "good"),
    Chip(4, "good"),
    Chip(5, "bad"),
]

a, b = chips[4], chips[2]
print(test(a, b))

def find_one_good_chip(chips):
    correct_chips = chips[:]
    
    while len(correct_chips) > 1:
        next_round = []
        i = 0

        while i + 1 < len(correct_chips):
            a = correct_chips[i]
            b = correct_chips[i + 1]

            a_says_b, b_says_a = test(a, b)
            if a_says_b and b_says_a:
                next_round.append(a)

            i += 2 # discard both

        # carry over if odd chips
        if i < len(correct_chips):
            next_round.append(correct_chips[i])

        correct_chips = next_round

    if correct_chips:
        return correct_chips[0]

    return None

def classify_chips(chips, good_chip):
    result = {}

    for chip in chips:
        if chip == good_chip:
            result[chip.id] = "good"
        else:
            good_says, _ = test(good_chip, chip)
            result[chip.id] = "good" if good_says else "bad"

    return result

def identify_chips(chips):
    good_chip = find_one_good_chip(chips)

    if good_chip is None:
        print("no good chip found")
        return None

    return classify_chips(chips, good_chip)

print(identify_chips(chips))

(False, False)
no good chip found
None


In [22]:
"""
recurrence
"""
def find_one_good_chip_r(chips):
    if len(chips) == 1:
        return chips[0]

    next_round = []
    i = 0
    while i+1 < len(chips):
        a, b, = chips[i], chips[i+1]
        a_says_b, b_says_a = test(a, b)
        
        if a_says_b and b_says_a:
            next_round.append(a)

        i += 2

    if i < len(chips):
        next_round.append(chips[i])

    return find_one_good_chip_r(next_round)

In [24]:
def classify_all_chips(chips, good_chip):
    result = {}
    for chip in chips:
        if chip == good_chip:
            result[chip.id] = "good"
        else:
            good_says, _ = test(good_chip, chip)
            result[chip.id] = "good" if good_says else "bad"
    return result

def identify_all_chips(chips):
    good_chip = find_one_good_chip_r(chips)
    return classify_all_chips(chips, good_chip)

In [26]:
"""
Mongo Matrix
A[i,j]+A[i+1,j+1]≤A[i,j+1]+A[i+1,j]

m x n matrix where every 2x2 along with its adjacent
top-left corner + bottom-right corner <= top-right corner + bottom-left corner
"""

def is_monge(A):
    m = len(A)
    n = len(A[0])
    violations = []

    for i in range(m - 1):
        for j in range(n - 1):
            left = A[i][j] + A[i + 1][j + 1]
            right = A[i][j + 1] + A[i + 1][j]
            if left > right:
                violations.append((i, j, left, right))
    return violations

A = [
    [37, 23, 22, 32],
    [21,  6,  7, 10],
    [53, 34, 30, 31],
    [32, 13,  9,  6],
    [43, 21, 15,  8]
]

violations = is_monge(A)

if not violations:
    print("Matrix is Monge")
else:
    print("Matrix is NOT Monge")
    print("Violating 2x2 blocks:")
    for v in violations:
        i, j, left, right = v
        print(f"Rows {i}-{i+1}, Cols {j}-{j+1}: {left} > {right}")

# 22 row 0, column 3 is in violation

Matrix is NOT Monge
Violating 2x2 blocks:
Rows 0-1, Cols 1-2: 30 > 28


In [27]:
"""
SMAWK =
Schieber
Moran
Aggarwal
Wilber
Klawe

SMAWK is a linear-time algorithm for finding all row minima in a Monge array, made possible by the fact that row minima never move left.

Eliminates columns that can never be minima
Recursively solves a smaller problem
Fills in the skipped rows using the monotonicity

SMAWK repeatedly:
compares pairs of rows
removes columns that lose comparisons
guarantees no removed column could be a minimum
This works only because of Monge monotonicity.
Without it, removing a column could destroy correctness.

FULL MATRIX
Columns →    1   2   3   4   5   6   7
Rows ↓
1            .   .   X   .   .   .   .
2            .   .   .   X   .   .   .
3            .   .   .   X   .   .   .
4            .   .   .   .   .   X   .
5            .   .   .   .   .   X   .
6            .   .   .   .   .   .   X

KEEP ONLY EVEN ROWS (temporarily) (DIVIDE AND CONQUER):
Columns →    1   2   3   4   5   6   7
Rows ↓
2            .   .   .   X   .   .   .
4            .   .   .   .   .   X   .
6            .   .   .   .   .   .   X


f(2) = 4
f(4) = 6
f(6) = 7

BRING BACK ODD ROWS:
Columns →    1   2   3   4   5   6   7
Rows ↓
1            .   .   ?   ?   ?   ?   ?
2            .   .   .   X   .   .   .
3            .   .   ?   ?   ?   ?   ?
4            .   .   .   .   .   X   .
5            .   .   ?   ?   ?   ?   ?
6            .   .   .   .   .   .   X

row 1 scans → [1 .. 4]
row 3 scans → [4 .. 6]
row 5 scans → [6 .. 7]

Basic row-recursion only →
O(m+nlogm)

Full SMAWK with column elimination →
O(m+n)
"""

'\nSMAWK =\nSchieber\nMoran\nAggarwal\nWilber\nKlawe\n\nSMAWK is a linear-time algorithm for finding all row minima in a Monge array, made possible by the fact that row minima never move left.\n\nEliminates columns that can never be minima\nRecursively solves a smaller problem\nFills in the skipped rows using the monotonicity\n\nSMAWK repeatedly:\ncompares pairs of rows\nremoves columns that lose comparisons\nguarantees no removed column could be a minimum\nThis works only because of Monge monotonicity.\nWithout it, removing a column could destroy correctness.\n\nFULL MATRIX\nColumns →    1   2   3   4   5   6   7\nRows ↓\n1            .   .   X   .   .   .   .\n2            .   .   .   X   .   .   .\n3            .   .   .   X   .   .   .\n4            .   .   .   .   .   X   .\n5            .   .   .   .   .   X   .\n6            .   .   .   .   .   .   X\n\nKEEP ONLY EVEN ROWS:\n'